# Underwriting Guidelines Project

The goal of this Jupyter Notebook is to develop a logical framework capable of determining whether an arbitrary underwriter has the authority to quote an insurance submission. This problem is approached using principles from object-oriented programming, data structures, and algorithm design, with an emphasis on time and space efficiency for guideline lookups and authority verification. 

### Objective

The core task is to design an `is_authorized()` method. Given an arbitrary underwriter and submission, it should return whether the submission falls within the underwriter’s authority to quote, and if not, provides the reason why. It is written in the form `underwriter.is_authorized(submission)`. Later on we will randomly generate submissions and test the capability of an underwriter, and an underwriting manager.

#### Class Design

To achieve this, two main classes are defined:

1. Underwriter: representing the underwriter’s authority parameters. Each underwriter has:
- A name
- A maximum premium authority limit. This is typically based on modified premium.
- Minimum and maximum experience rating modification (XMOD) bounds
- A list of territories they are permitted to write business in
- A managerial level indicator, since certain class codes require managerial authority to approve. If their managerial level is 1 then they are a manager.

Method: `is_authorized()`: This method evaluates a submission against these rules, checking:
- If the submission’s modified premium is within authority
- If the experience rating (XMOD) is within allowable limits
- If all submission territories are authorized
- If the submission contains class codes requiring managerial referral

The method will return false at the first failure, checking first modified premium, Xmod, territory, and lastly class code. This order was selected because the complexity of the subsequent verficiation is larger. For instance, the premium verification is a simple check, `submission.modified_premium > underwriter.premium_authority` whereas the class code authority performs a lookup into the underwriting guidelines dictionary, and a lookup into the incidental dictionary, and performs this lookup for each class code in the submission. For scalability, reducing the algorithmic complexity is important. If there were thousands of submissions, we would be performing many unnecessary checks. As a sidenote, this is referred to as the partial authorization function. One can also define a 'full' authorization function that returns every single failure. 

2. Submission: representing an account or risk being quoted. A submission includes:
- A name
- A dictionary of class codes and their exposure basis
- An Xmod
- A list of territories

The submission class also computes:
- Manual Premium: The sum of `exposure * class_rate`
- Modified Premium: `manual_premium * Xmod`
- The share of total manual premium attributable to each class code

#### Data Structure Design

##### Guidelines
Dictionaries were developed to define the underwriting rules per class code. Each class code is designated as either standard or managerial, with an optional additional dictionary for incidental threshold percentages. This allows flexibility for classes that require referral only when their exposure share exceeds a defined threshold. 

`
managerial_guideline_dictionary = {}
incidental_threshold_guidelines_dictionary = {}
`

Python dictionaries provide average-case $O(1)$ lookups. The theoretical worst case is $O(n)$ but this is effectively a non-issue for normal workloads. In practice, dict lookups are constant-time for our purposes.

The incidental threshold guidelines is technically able to be combined with the managerial guideline dictionary to reduce storage, but the access structure becomes fairly awkward, hence why an additional dictionary for incidental thresholds was selected. For instance, in a combined guideline dictionary the data structure could look like this: `{8810: [1, 0.1]}`. In this hypothetical, class code 8810 is a managerial class code, and is acceptable incidentally up to a threshold of 10%. However to access either of these values `guideline[8810]` results in a list structure. Access is position dependent, `flag = guideline[cc][0]` and `threshold = guideline[cc][1]`. The code readability suffers at call sites. Python lists also don't have a `.get()` method like dictionaries do which makes defaulting for values potentially not in the guideline more difficult if for instance, a glitched class code were to appear. 

The code in practice looks like this: `submission.class_manual_percent(cc) > incidental_threshold_guidelines_dictionary.get(cc, 0.0)`. The tradeoff in additional storage for the ability to apply `.get` with a default value is why this was method was preferred over the listed guideline method. The two-dictionary layout trades a negligible amount of memory for clean defaults, simpler code, and easier maintenance, which improves correctness and readability without meaningful performance loss.

In the next lines of code, the WCIRB pure premium rates are imported as a base rate dictionary and the managerial/incidental guidelines dictionaries are populated. These guidelines are randomized, with a 10% chance of each class code being a managerial class code, and the incidental thresholds being set according to $ U \sim \text{unif}(0.05,0.25)$ rounded to 4 decimal places. A brief snapshot of the guidelines will also be displayed.

In [1]:
import csv
import random
import pandas as pd

pure_premium_rates = {} #Start with an empty dictionary, we will populate this with the WCIRB pure premium rates as class code - pure premium rate pairs.
with open("approved_09012025_pure_premium_rates.csv", newline = "") as pure_premiums:
    reader = csv.reader(pure_premiums)
    for row in reader:
        class_code, rate = row
        pure_premium_rates[class_code] = float(rate)

managerial_guideline_dictionary = {}
incidental_threshold_guidelines_dictionary = {}

random.seed(42)
for class_code in pure_premium_rates.keys():
    # 10% chance of being a managerial class code
    is_managerial = random.random() < 0.10
    managerial_guideline_dictionary[class_code] = 1 if is_managerial else 0
    if is_managerial:
        threshold = round(random.uniform(0.05, 0.25), 4)  # fraction, not %
        incidental_threshold_guidelines_dictionary[class_code] = threshold

In [2]:
print("Managerial guideline sample:")
for k, v in list(managerial_guideline_dictionary.items())[:10]:
   print(k, v)

Managerial guideline sample:
0005 0
0016 1
0034 0
0035 0
0036 0
0038 0
0040 1
0041 1
0042 0
0045 1


In [3]:
print("\nIncidental threshold sample:")
for k, v in list(incidental_threshold_guidelines_dictionary.items())[:10]:
   print(k, f"{v:.2%}")


Incidental threshold sample:
0016 10.50%
0040 13.44%
0041 9.37%
0045 8.98%
0172 21.12%
1330 6.93%
2014 9.56%
2081 9.66%
2584 11.31%
2881 5.94%


In the following lines of code, we define the underwriter and submission class, as well as the methods such as the `is_authorized()`.

In [4]:
class Underwriter():
    def __init__(self, name, premium_authority, xmod_min_authority, xmod_max_authority, territory_authority, managerial_level):
        self.name = name
        self.premium_authority = premium_authority
        self.xmod_min_authority = xmod_min_authority
        self.xmod_max_authority = xmod_max_authority
        self.territory_authority = territory_authority  #List of states as a set for O(1) look up. This scales with a large number of territories.
        self.managerial_level = managerial_level
# -------------------------------------------------------------------------
    def is_authorized(self, submission):
        violations = []
        if submission.modified_premium > self.premium_authority:
            return False, "Referral: Premium"
        if not (self.xmod_min_authority <= submission.xmod <= self.xmod_max_authority):
            return False, "Referral: Xmod"
        for territory in submission.territory:
            if territory not in self.territory_authority:
                return False, f"Referral: Territory ({territory})"
            
            # Managerial class referral logic
 # --- Managerial bypass ---
        if self.managerial_level == 1:
            # Managers can approve any class distribution
            return True, "Account is within underwriting authority"
            
        violating = []
        for cc in submission.exposure_basis.keys():
            is_managerial = managerial_guideline_dictionary.get(cc, 0) == 1
            if not is_managerial:
                continue

            pct = submission.class_manual_percent(cc)  # fraction 0..1
            # If no threshold provided for a managerial class, treat as non-incidental allowed = 0% (i.e., always referral)
            thr = incidental_threshold_guidelines_dictionary.get(cc, 0.0)
            if pct > thr:  # non-incidental managerial operation
                violating.append(f"{cc} ({pct:.1%} > {thr:.0%})")

        if violating:
            return False, "Referral; Class Codes: " + ", ".join(violating)
        
        return True, "Account is within underwriting authority"

# ------------------------------------------------------------------------------------------
class Submission():
    def __init__(self, name, exposure_basis, xmod, territory):
        self.name = name
        self.exposure_basis = exposure_basis  #Dictionary of class codes & exposure, class code is the key with the exposure as the value. 
        self.xmod = xmod
        self.territory = territory  #State code
        self.manual_premium = self.calc_manual_premium() #calculate the aggregate manual premium accross class codes & exposures
        self.modified_premium = self.calc_modified_premium() #calculate modified premium

    def __repr__(self):
        exposures_str = ", ".join(
            [f"{cc}: {exp:,}" for cc, exp in self.exposure_basis.items()]
        )
        manual_prem = self.manual_premium
        modified_prem = self.modified_premium
        return (
            f"Submission(name='{self.name}', "
            f"xmod={self.xmod:.2f}, "
            f"territory={self.territory}, "
            f"manual_premium={manual_prem:,.2f}, "
            f"modified_premium={modified_prem:,.2f}, "
            f"exposures={{ {exposures_str} }})"
        )
        
    def calc_manual_premium(self): 
        # Sum of (exposure * class rate) across all class codes
        total = 0
        for class_code, exposure in self.exposure_basis.items():
            rate = pure_premium_rates.get(class_code, 0)
            total += exposure * rate/100
        return total
        
    def calc_modified_premium(self):
        # Manual premium multiplied by xmod
        return self.manual_premium * self.xmod
        
    def class_manual_percent(self, class_code):
    #Return the manual premium % contribution for a given class code.
        if class_code not in self.exposure_basis:
            raise ValueError(f"Class code {class_code} not in submission.")
        
        rate = pure_premium_rates.get(class_code, 0)
        exposure = self.exposure_basis[class_code]
        class_premium = exposure * rate/100

        manual_premium = self.calc_manual_premium()  # compute on demand
        if manual_premium == 0:
            return 0.0

        return class_premium / manual_premium
        
    def calc_managerial_authority(self):
        pass

class Guideline():
    def __init__(self, class_code, managerial_level, incidental_managerial_threshold = None):
        self.class_code = class_code #The class code
        self.managerial_level = managerial_level #problematic class codes. 
        self.incidental_managerial_threshold = incidental_managerial_threshold
        #incidental_managerial_threshold will be determined IF it is a managerial class code.
        

To test these, we can define an underwriter and an underwriting manager.

The underwriter will have a premium authority up to 200,000, Xmods between 60% & 140%, authority in California, and a managerial level of 0 i.e. not a manager.

The manager will have a premium authority up to 1,000,000, Xmods between 40% and 200%, authority in California & AOS, and a managerial level of 1.

In [5]:
uw = Underwriter('Underwriter', premium_authority = 200000, xmod_min_authority = 0.6, xmod_max_authority = 1.4,territory_authority = ['CA'], managerial_level = 0)
mg = Underwriter('Manager', premium_authority = 1000000, xmod_min_authority = 0.4, xmod_max_authority = 2, territory_authority = ['CA', 'AOS'], managerial_level = 1)

We will generate random businesses with arbitrary parameters and verify that our underwriters are able to make a decision on them.

For simplicity, this initial implementation will focus on California and All Other States (AOS). The selected pure premium rates are based on WCIRB workers’ compensation class codes, though this framework can be extended to include NCCI and other independent rating bureaus if needed.

The `generate_random_submission()` function will randomly select an exposure basis composed of one to four class codes, assign a random experience modification factor (Xmod), and choose territories randomly between “CA” and “AOS.”

In [6]:
def generate_random_submission(possible_states, possible_class_codes):
    # Randomly pick exposure basis: 1–4 class codes
    num_classes = random.randint(1, 4)
    class_codes = random.sample(possible_class_codes, num_classes)
    exposure_basis = {cc: random.randint(50_000, 2_000_000) for cc in class_codes}
    xmod = round(random.uniform(0.25, 1.75), 2)
    # Random territory list (e.g., 1–3 states)
    territory = random.sample(possible_states, random.randint(1, 2))
    name = f"Business_{random.randint(1000, 9999)}"
    return {
        "name": name,
        "exposure_basis": exposure_basis,
        "xmod": xmod,
        "territory": territory,
    }

In [7]:
generate_random_submission(possible_states = ["CA", "AOS"], possible_class_codes = list(pure_premium_rates.keys()))

{'name': 'Business_2983',
 'exposure_basis': {'4983': 209376,
  '9507': 1291719,
  '8292': 1498323,
  '3169': 1791834},
 'xmod': 1.19,
 'territory': ['AOS']}

Above is an example of a randomly generated business. 

In the following code, we define a list called `subs` containing a list of random submissions. We will iterate `is.authorized()` over the list and compile the outputs into a pandas dataframe to analyze the results.

In [8]:
random.seed(42)
subs = []
for _ in range(20):
    attrs = generate_random_submission(possible_states = ["CA", "AOS"], possible_class_codes = list(pure_premium_rates.keys()))
    sub = Submission(**attrs)  # unpack dict into class init
    subs.append(sub)

## Testing

In [9]:
def to_row(sub, uw):
    authorized, reason = uw.is_authorized(sub)
    # Normalize territory to a string for display
    terr = sub.territory if isinstance(sub.territory, str) else ",".join(sub.territory)
    return {
        "name": sub.name,
        "territory": terr,
        "xmod": sub.xmod,
        "manual_premium": round(sub.manual_premium, 2),
        "modified_premium": round(sub.modified_premium, 2),
        "authorized": authorized,
        "failure_reason": reason if not authorized else "",
        "n_classes": len(sub.exposure_basis),
    }

uw_authority = [to_row(sub, uw) for sub in subs]
uw_authority_df = pd.DataFrame(uw_authority)
display(uw_authority_df)

,name,territory,xmod,manual_premium,modified_premium,authorized,failure_reason,n_classes
0,Business_2679,CA,0.66,72552.51,47884.66,True,,1
1,Business_4811,CA,0.30,6730.80,2019.24,False,Referral: Xmod,1
2,Business_5557,"CA,AOS",1.32,46792.60,61766.23,False,Referral: Territory (AOS),1
3,Business_3547,"AOS,CA",0.49,5393.77,2642.95,False,Referral: Xmod,1
4,Business_6635,AOS,0.39,91979.70,35872.08,False,Referral: Xmod,2
5,Business_6925,"CA,AOS",1.71,102219.41,174795.19,False,Referral: Xmod,3
6,Business_2654,"CA,AOS",0.59,85109.60,50214.67,False,Referral: Xmod,2
7,Business_9751,"CA,AOS",0.56,89777.77,50275.55,False,Referral: Xmod,4
8,Business_1916,AOS,1.73,88498.14,153101.78,False,Referral: Xmod,2
9,Business_6155,CA,0.85,47985.06,40787.30,True,,2


In the above output, we can see that the underwriter does not have the authority to make a decision on the majority of incoming risks. Out of 20 submissions, there are only 3 which a decision is able to be made on without managerial approval. The most common one appears to be due to the Xmod, which does make sense as $X_{\text{mod}} \sim \text{unif}(0.25,1.75)$ and $P(0.6 < X_{\text{mod}} < 1.4) = \frac{1.4-0.6}{1.75-0.25} = \frac{8}{15} \approx 0.53$.

There is only 1 risk which failed at class code 5633, because there was authority only up to 13% for that exposure.

In [10]:
# Only in authority
display(uw_authority_df[uw_authority_df["authorized"]])

,name,territory,xmod,manual_premium,modified_premium,authorized,failure_reason,n_classes
0,Business_2679,CA,0.66,72552.51,47884.66,True,,1
9,Business_6155,CA,0.85,47985.06,40787.30,True,,2
18,Business_3103,CA,0.98,88914.07,87135.79,True,,1


Now, we'll switch it to the underwriting manager to see what they have authority on

In [11]:
mg_authority = [to_row(sub, mg) for sub in subs]
mg_authority_df = pd.DataFrame(mg_authority)
display(mg_authority_df)

,name,territory,xmod,manual_premium,modified_premium,authorized,failure_reason,n_classes
0,Business_2679,CA,0.66,72552.51,47884.66,True,,1
1,Business_4811,CA,0.30,6730.80,2019.24,False,Referral: Xmod,1
2,Business_5557,"CA,AOS",1.32,46792.60,61766.23,True,,1
3,Business_3547,"AOS,CA",0.49,5393.77,2642.95,True,,1
4,Business_6635,AOS,0.39,91979.70,35872.08,False,Referral: Xmod,2
5,Business_6925,"CA,AOS",1.71,102219.41,174795.19,True,,3
6,Business_2654,"CA,AOS",0.59,85109.60,50214.67,True,,2
7,Business_9751,"CA,AOS",0.56,89777.77,50275.55,True,,4
8,Business_1916,AOS,1.73,88498.14,153101.78,True,,2
9,Business_6155,CA,0.85,47985.06,40787.30,True,,2


The only authority failures are for Xmods which happen to be extremely low. They have authority in all territories, all class codes, and premium up to $1m. The results align with the initial parameters set in.

# Conclusion

This project successfully developed a logical and scalable framework for automating underwriting authority checks using object-oriented programming principles. The implementation included both data and class structures that mirror real-world underwriting logic, ensuring each component remains modular, interpretable, and computationally efficient.

1. Class Design and Architecture

We defined a robust schema for Underwriter, Submission, and Guideline objects.
- Underwriter encapsulates decision authority (premium, XMOD, territory, managerial level).
- Submission models real account characteristics (class codes, exposures, territories, XMOD, and calculated premiums).
- Global dictionaries store guideline parameters (managerial flags and incidental thresholds), ensuring clarity, flexibility, and constant-time lookups.

2. Simulation and Testing
- Generated 20 randomized submissions to test the authority logic across various exposure mixes, XMODs, and territories.
- Each case was evaluated using underwriter.is_authorized(submission), confirming that the system properly identified whether an account was within authority and, when not, returned the correct referral reason.
- Results were visualized in a pandas DataFrame to ensure no missed cases or logic inconsistencies.

The resulting system demonstrates that underwriting guidelines can be efficiently modeled through structured, object-oriented logic and dictionary-based data management. The approach balances algorithmic efficiency, transparency, and real-world adaptability—providing a foundation that can scale to more complex multi-line authority frameworks or integrate seamlessly with a production underwriting platform.